In [2]:
import tweepy
import csv

# Replace the values in the following lines with your own Twitter API keys
api_key = "ufmmiP6WPrf88KFWBucUYqyqj"
api_secret = "VC5lt1Wa6Y4TDsMByVSzRe5iB36XXNMdFrD99yibpTKt4rHtXS"
access_token = "1483027022645706754-4JCnTuUQ8FfUMBS8MOre8cufJBK2O6"
access_token_secret = "5y0pofXx8zwSNJwCWaPFRK04KYli2E3MAsP29lpsB2tZO"

# Authenticate to Twitter API
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)

# Create API object
api = tweepy.API(auth)

# Search for tweets in the specified location
tweets = tweepy.Cursor(api.search_tweets,q="",
              geocode="41.0082,28.9784,1km",
              lang="en",
              tweet_mode='extended').items(1000)

# Extract only the text of tweets in English language
english_tweets = [tweet.full_text for tweet in tweets if tweet.lang == "en"]

# Write the extracted tweets to a CSV file
with open("tweets_english.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Latitude", "Longitude","Text"])
    for tweet in english_tweets:
        if tweet.coordinates:
            # Write the tweet text, latitude, and longitude to the CSV file
            writer.writerow([tweet.coordinates["coordinates"][1], tweet.coordinates["coordinates"][0],tweet.text])


AttributeError: 'str' object has no attribute 'coordinates'

In [7]:
import tweepy
import csv

# Replace the values in the following lines with your own Twitter API keys
api_key = "ufmmiP6WPrf88KFWBucUYqyqj"
api_secret = "VC5lt1Wa6Y4TDsMByVSzRe5iB36XXNMdFrD99yibpTKt4rHtXS"
access_token = "1483027022645706754-4JCnTuUQ8FfUMBS8MOre8cufJBK2O6"
access_token_secret = "5y0pofXx8zwSNJwCWaPFRK04KYli2E3MAsP29lpsB2tZO"

# Authenticate to Twitter API
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)

# Create API object
api = tweepy.API(auth)

# Search for tweets in the specified location
tweets = tweepy.Cursor(api.search_tweets,q="",
              geocode="41.0082,28.9784,1km",
              lang="en",
              tweet_mode='extended').items(10000)

# Extract the text and coordinates of tweets in English language
english_tweets = [(tweet.full_text, tweet.coordinates) for tweet in tweets if tweet.lang == "en" and tweet.coordinates is not None]

# Write the extracted tweets, latitude, and longitude to a CSV file
with open("tweets_english.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Latitude", "Longitude","Tweet"])
    for tweet, coord in english_tweets:
        lat, lon = coord["coordinates"]
        writer.writerow([lat, lon, tweet])


In [8]:
import csv
import re

# The input CSV file
input_file = "tweets_english.csv"

# The output CSV file
output_file = "output_english.csv"

# Regular expression pattern to match URLs
url_pattern = re.compile(r"(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?")

# Read the input CSV file
with open(input_file, "r") as f_input:
    reader = csv.reader(f_input)
    header = next(reader)
    text_column_index = header.index("Tweet")
    data = [row for row in reader]

# Split the text and URLs into separate columns
rows = []
for row in data:
    text = row[text_column_index]
    urls = re.findall(url_pattern, text)
    at_index = text.find("@")
    if at_index != -1:
        text_before_at = text[:at_index].strip()
        text_after_at = text[at_index:].strip()
    else:
        text_before_at = text
        text_after_at = ""
    for url in urls:
        text_before_at = text_before_at.replace(url[0] + "://" + url[1] + url[2], "")
        text_before_at = re.sub(r'[^\w\s]', '', text_before_at) # remove symbols
        text_before_at = re.sub(r'#\w+', '', text_before_at) # remove hashtags
        rows.append(row[:text_column_index] + [text_before_at, text_after_at] + [url[0] + "://" + url[1] + url[2]] + row[text_column_index+1:])

# Write the output CSV file
with open(output_file, "w", newline="") as f_output:
    writer = csv.writer(f_output)
    writer.writerow(header + ["Text after @", "URL"])
    writer.writerows(rows)
